# Streamlit App - Gender Classification, Facemask Detection, and Facemask Removal


# Install Streamlit

In [ ]:
! pip install streamlit
! pip install cvlib

In [ ]:
# !unzip /content/gender_detection.model.zip

Create a app.py file for sreamlit to run

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%writefile app.py
import numpy as np
import streamlit as st
import tensorflow as tf
import cvlib as cv
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
import numpy as np
import tensorflow as tf
import joblib
from loadclass import Load_model
from vgg_model import VGG19_model
import cv2
from PIL import Image, ImageOps

# Gender Classification
def image_preprocessing(image):
  input_image = image
  face, confidence = cv.detect_face(input_image)
  start_X, start_Y, end_X, end_Y = face[0]
  resize_image = cv2.resize(input_image[start_Y:end_Y,start_X:end_X],(96,96))
  resize_image = resize_image.astype("float")/ 255.0
  img_array = img_to_array(resize_image)
  final_image = np.expand_dims(img_array, axis=0)
  return final_image

def predict(preprocessed_image):
  my_model = tf.keras.models.load_model('/content/drive/MyDrive/cmp258/temp/gender_detection.model')
  labels = ["Man","Woman"]
  prediction = my_model.predict(preprocessed_image)[0]
  Predicted_label = labels[np.argmax(prediction)]
  return Predicted_label

def gender_classification(opencv_image):
    st.subheader("Gender Classificaiton")
    preprocessed_image = image_preprocessing(opencv_image)
    prediction = predict(preprocessed_image)
    return prediction

# Facemask Detection
def mask_predict(img):
    st.subheader("Mask Detection")
    # model = joblib.load('/content/drive/MyDrive/cmp258/temp/part-a-model-2.sav')
    gcs_path = 'gs://face-mask-detection-model/part-a-model.sav'
    model = joblib.load(tf.io.gfile.GFile(gcs_path, 'rb'))
    # if type(img) == str:
    #     img = cv2.imread(img)
    img = cv2.resize(img,(200,200))
    img = img / 255
    if model.predict(np.array([img]))[0] > 0.5:
        predict = 1 
        predictString = "unmasked"
    else:
        predict = 0
        predictString = "masked"
      
    st.write("Facemask Detection: This person is: ",  predictString)
    return predict

# Facemask Removal
def facemaskRemoval(img):
      mask_g = "/content/drive/MyDrive/cmp258/temp/MaskG"
      face_g = "/content/drive/MyDrive/cmp258/temp/FaceG"
      face_D_region = "/content/drive/MyDrive/cmp258/temp/face_D_region"
      face_D_whole = "/content/drive/MyDrive/cmp258/temp/face_D_whole"
      mask_g_model = tf.keras.models.load_model(mask_g, compile=False)
      face_g_model= tf.keras.models.load_model(face_g, compile=False)
      face_D_whole_model= tf.keras.models.load_model(face_D_whole, compile=False)
      face_D_region_model= tf.keras.models.load_model(face_D_region, compile=False)
      vgg_model = VGG19_model()
      try:
        test = Load_model(mask_g_model, face_g_model, mask_checkpoint_dir="/content/drive/MyDrive/cmp258/temp/Checkpoints/mask32_checkpoints", face_checkpoint_dir="/content/drive/MyDrive/temp/Checkpoints/face_checkpoints")
        test.load()
        # st.image(img,channels="BGR")
      except:
        print("some error")
      img = test.one_predict(img)

# Define the Image function
def predictImage(uploaded_file):
    file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    opencv_image = cv2.imdecode(file_bytes, 1)
    #call pooja's function/model to check if mask exist
    if (mask_predict(opencv_image) == 1):
      st.write("Gender Classification: This person is a ",  gender_classification(opencv_image))
    else:
      facemaskRemoval(opencv_image)
      removed_mask = cv2.imread('unmasked.jpeg')
      #finally call the gender classificaiton function
      st.write("Gender Classification: This person is a ",  gender_classification(opencv_image))

# Create the Application
st.title('Gender Classification, Facemask Detection, and Facemask Removal')
uploaded_file = st.file_uploader("Choose a image file", type="jpeg")
input = st.button('Predict')
# Generate the prediction based on the users input
if input:
    st.image(uploaded_file, channels="BGR",width=150)
    predictImage(uploaded_file)

Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

2022-05-22 23:45:37.069 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.719s
your url is: https://green-suns-wonder-34-148-161-35.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.148.161.35:8501

2022-05-22 23:45:59.735477: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "NOT_FOUND: Error executing an HTTP request: HTTP response code 404".
2022-05-22 23:46:04.379748: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-22 23:46:15.231 Detecting that an object or model or tf.train.Checkpoint is being deleted with unrestored values. See the following logs for 